# Numpy 的多维数组

numpy 最主要的特性就是提供了这个名为 `ndarray` 的多维数组类。其采用 C 实现，提供了 Python 接口。

ndarray 可以通过构造函数 `np.ndarray` 来创建，也可以使用一个简写的构造函数 `np.array`。
不过，前者会创建一个空的数组，后者则额外接收初始化数据。

在使用之前，先了解一些基本概念：

1. ndarray 不是矩阵。

ndarray 是多维数组，虽然当构造为 2 维数组时看起来很像矩阵，但其运算法则不是。
在 ndarray 的计算中，你会遇到名为 [广播]() 的现象，这对于一些数据处理来说有一定作用。

2. ndarray 中的元素具有同一类型。

整个 numpy 库都是用 C 实现的，只是提供了 Python 接口罢了。ndarray 也是同样。
其中的元素都必须有 C 类型，`int`, `float` 等，而不是像 Python 那样，各个类型可以随意转换
（Python 也是用 C 实现的，在 C 中， Python 对象其实都是 `PyObject` 结构体。

在构造函数 `ndarray` 中，
默认的数据类型是 `float`。这些类型名称可以在 numpy 的顶级模块找到，如 `np.float64` 等。
numpy 中的大多数函数都接受 `dtype` 参数以设置类型。

对于构造函数 `np.array`，其会自动分析传入的数据，将其设置为相对应的类型： `int` -> `np.int`, `float` -> `np.float`。

In [1]:
import numpy as np

> `np` 是导入 numpy 时的惯用别名，其他常用工具如 `pandas` 常用 `pd`, `matplotlib.pyplot` 则常简写为 `plt`。

## 构造多维数组

让我们试试创建一个二维数组。

可以向构造函数传入 Python 中的列表/元组等容器元素，将其中的数据转化为 ndarray 中的元素，如：

In [2]:
np.ndarray(shape=(2, 2), dtype=np.float64)

array([[ 0.00000000e+000,  0.00000000e+000],
       [ 0.00000000e+000, -8.92580403e-311]])

使用 `np.ndarray` 时，根据设置的 shape，dtype 参数，将这个数组构造为 $2\times2$ 的 64 位浮点数数组。
如你所见，其内容是空的，值非常接近于 0.（可能是 $n\times10^{-300}$ 这样的值）。

使用 `np.array` 时，需要传入初始化数据，但不能设置 shape 参数了， shape 将会根据传入的参数自动设置：

In [3]:
np.array([
    [1, 2],
    [3, 4],
], dtype=np.float)

array([[1., 2.],
       [3., 4.]])

当我们传入一个嵌套的列表时，numpy 将其当作一个维度大于 1 的数组处理，嵌套层级越多，维度越高。注意这里设置了 `dtype=np.float` 参数，这是因为我们传入的列表中的元素是 Python 的 int 类型，numpy 会将其转化为 C 的 int。如果不设置 `dtype` 的话，数组中的数据类型就是整数，无法用来模拟数学运算中的实数了。

当然，如果你本来就打算处理整数，例如处理图片，的话，就可以设置为整数。还有 `np.uint`, `npuint8` 等多种类型可用。

总之，在不同的使用场景上，使用合适的数据类型。

### 常见参数名

上面提到了，numpy 中许多函数都接收 `dtype` 参数来设置类型。此外，也有需要常用的参数，例如 `shape`, `axis` 等。

- `shape`: 一个元组，用于设置数组的大小与形状。元组的长度决定了数组的维度，在每一位上的值则决定了数组在这一维上的长度。
- `axis`：0 或 1，

## 多维数组取值或切片

ndarray 实现了一系列操作符重载，要取出某个值或者取出切片，和普通列表一样使用 `[]` 运算府。

> 重载对象的 `__getitem__` 方法。

当维度只有 1 时， ndarray 表现得和普通列表没什么两样。

In [4]:
A = np.array([1, 2, 3, 4, 5, 6])

取出一个元素：

In [5]:
A[0]

1

切片：

In [6]:
A[3:]

array([4, 5, 6])

当维度增加后，ndarray 的 `[]` 运算符就可以接收元组了，元组中从左到右，分别是作用于从低到高维度的索引或切片。
也可以接收 int 或 slice 对象，传入的索引长度不够也会转化为完整的元组，其他值默认以 `:` 填充。

> `:` 是用于便利地构造 slice 对象的语法糖，但只在 `[]` 操作符中解析。

In [7]:
A = np.array([
    [
        [111, 112, 113],
        [121, 122, 123],
        [131, 132, 133],
    ],
    [
        [211, 212, 213],
        [221, 222, 223],
        [231, 232, 233],
    ],
    [
        [311, 312, 313],
        [321, 322, 323],
        [331, 332, 333],
    ],
])

当我们传入 0 时，其效果和传入 `0, :, :` 是一样的，会选择第一维坐标为 `0` 的切片：

In [8]:
A1 = A[0]
A1

array([[111, 112, 113],
       [121, 122, 123],
       [131, 132, 133]])

In [9]:
A[0, :, :]

array([[111, 112, 113],
       [121, 122, 123],
       [131, 132, 133]])

传入 `0, 0` 时，就相当于 `0, 0, :`

In [10]:
A11 = A[0, 0]
A11

array([111, 112, 113])

传入 slice 以返回一个范围

In [11]:
A[0, :2, 0]

array([111, 121])

接下来，要搞清楚一个问题：**选出的元素是值，还是引用？**

这关系到对选出值的修改会不会影响到源数组的问题。

对于切片，看来是 **不会影响**。

In [12]:
A11 = np.array([999, 998, 997])

In [13]:
A

array([[[111, 112, 113],
        [121, 122, 123],
        [131, 132, 133]],

       [[211, 212, 213],
        [221, 222, 223],
        [231, 232, 233]],

       [[311, 312, 313],
        [321, 322, 323],
        [331, 332, 333]]])

对于元素，也 **不会影响**。

In [14]:
Ax = A[0, 0, 0]
Ax = 0
A

array([[[111, 112, 113],
        [121, 122, 123],
        [131, 132, 133]],

       [[211, 212, 213],
        [221, 222, 223],
        [231, 232, 233]],

       [[311, 312, 313],
        [321, 322, 323],
        [331, 332, 333]]])

接下来我们试试直接赋值：

In [15]:
A[0, 0, 0] = 999

In [16]:
A

array([[[999, 112, 113],
        [121, 122, 123],
        [131, 132, 133]],

       [[211, 212, 213],
        [221, 222, 223],
        [231, 232, 233]],

       [[311, 312, 313],
        [321, 322, 323],
        [331, 332, 333]]])

In [17]:
A[0, 0, :] = np.array([888, 777, 666])
A

array([[[888, 777, 666],
        [121, 122, 123],
        [131, 132, 133]],

       [[211, 212, 213],
        [221, 222, 223],
        [231, 232, 233]],

       [[311, 312, 313],
        [321, 322, 323],
        [331, 332, 333]]])

这样就会影响到了，什么原理？

<!-- todo -->

总结一下，当选出的区域赋值给一个变量时，发生了复制行为，对选出部分的修改不会影响到源。
但当直接对选出部分赋值时，就会直接修改源了。

## 特殊的多维数组

numpy 提供了一系列常用多维数组/矩阵的构造方法，只要传入 shape，就会自动创建并将值初始化。

### zeros

零数组，传入 shape 参数。

In [18]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

In [19]:
# 零矩阵
np.zeros((5, 5))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

### eye

单位矩阵。传入维度参数 N，创建对应的二维数组。

如果额外传入列参数 M，则只保留 M 之前的列。

In [20]:
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [21]:
np.eye(3, 2)

array([[1., 0.],
       [0., 1.],
       [0., 0.]])

### ones

一数组：

In [22]:
np.ones((3, 3))

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

### 随机数组

创建指定 shape 的随机数组成的数组。 （0~1）

In [23]:
np.random.random((3, 3))

array([[0.94323611, 0.37440566, 0.78161759],
       [0.26326898, 0.86783822, 0.34688587],
       [0.41556548, 0.78485376, 0.52480358]])

在 `np.random` 子模块中，具有与标准库 `random` 命名相同的函数，用于以不同的随机数生成器生成数组。或者将原本操作单个数据的函数变为操作数组的函数。

### 指定起点终点以及总数的线性序列

用 `np.linspace` 创建具有线性规律的数组。
一般设置 `start`, `stop` 和 `num` 参数，确定范围以及点的数目。

可以设置 `axis` 参数，用来指定生成数组的轴。
`axis>0` 的情况下 `start` 和 `stop` 参数需要为元组，分别指定每一维度上的起始点。 axis 则指定依照多维空间中的哪一根坐标轴。

In [24]:
np.linspace(0, 1, 3)

array([0. , 0.5, 1. ])

In [25]:
np.linspace((0, 1), (3, 4), 3)

array([[0. , 1. ],
       [1.5, 2.5],
       [3. , 4. ]])

In [26]:
np.linspace((0, 1), (3, 4), 3, axis=1)

array([[0. , 1.5, 3. ],
       [1. , 2.5, 4. ]])

### 指定范围与步长的线性序列

使用 `np.arange`:

In [27]:
np.arange(0, 100, 25)

array([ 0, 25, 50, 75])

### 指数序列

和 linspace 类似，但各点之间按指数规律

$$
b^{x}
$$

再次映射，默认的基底为 10，可用 `base` 参数指定基底。

In [28]:
np.logspace(0, 24, 4, base=2, dtype=np.int)

array([       1,      256,    65536, 16777216])

In [29]:
[2**0, 2**8, 2**16, 2**24]

[1, 256, 65536, 16777216]

## 索引和切片

在一个数组中，如何找到一个元素，这是搜索问题；如何按条件取出部分数组，这是过滤问题；另外，也可以使用 Python 中的 int、slice、range 等结构，对数组进行切片。
这些操作都可以通过数组的 `operator[]` 进行（也就是 `__getitem__` 方法）。

另外， numpy 被设置为处理大量数据，当进行索引和切片时，其切出来的部分是原数组的引用，而不是值。
也就是说，numpy 数组的索引和切片得到的是视图而不是克隆，对视图的所有操作都会影响到源数组。

但是当这个视图被复制给一个变量时，就会发生复制行为了。

In [30]:
array = np.arange(1, 10, dtype=np.float64).reshape((3, 3))
array

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

In [31]:
# 复制
five = array[1, 1] # 5
five = 0.5
array

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

In [32]:
# 不复制
array[1, 1] = 0.5
array

array([[1. , 2. , 3. ],
       [4. , 0.5, 6. ],
       [7. , 8. , 9. ]])

In [33]:
# 不复制
array[2] = [70, 80, 90]
array

array([[ 1. ,  2. ,  3. ],
       [ 4. ,  0.5,  6. ],
       [70. , 80. , 90. ]])

In [34]:
# 复制
col1 = array[:, 0]
col1 = [100, 400, 700]
array

array([[ 1. ,  2. ,  3. ],
       [ 4. ,  0.5,  6. ],
       [70. , 80. , 90. ]])